In [26]:
import mysql.connector
import math 
db = mysql.connector.connect(
host = "localhost",
user = "root",
password ="",
database = "db"
)
cur = db.cursor()

In [27]:
import plotly.graph_objects as tab
import numpy as np 
from matplotlib import pyplot as plt 
import pygal

In [28]:
color = [ "red", "blue", "green", "yellow", "purple", "orange", "white", "black" ]

In [ ]:
def main( ):
    v = [];
    print('\t\t\t--------------------WELCOME TO INVESTMENT BOOK--------------------\n')
    while(True):
        print("\nENTER YOUR CHOICE :\n1.See Portfolio\n2.Show Weightage\n3.Investment insight\n4.Buy Stock\n5.Sell Stock\n6.Exit\n")
        choice = int( input( ) ) 
        v.sort( reverse = True )
        if choice == 1 :
            cur.execute( "SELECT * FROM INVESTMENT ORDER BY CURRENT_TOTAL DESC" )
            curr = cur.fetchall( )
            if( len( curr ) == 0 ) :
                print( "NO STOCK AVAILABLE" )
            else:
                stock_name = []
                quantity = []
                buy_value = []
                current_value = []
                total_buy = []
                total_sell = []
                for x in curr :
                    if( x[ 1 ] > 0 ) :
                        stock_name.append( x[ 0 ] ) 
                        quantity.append( x[ 1 ] )
                        buy_value.append( x[ 2 ] )
                        current_value.append( x[ 3 ] )
                        total_buy.append( x[ 4 ] )
                        total_sell.append( x[ 5 ] )
                fig = tab.Figure(data=[tab.Table( columnwidth=[3,2,3,3,3,3], header = dict( values =[ 'Stock_Name' , 'Quantity' , 'Buy_Value', 'Current_Total','Total Buy' , 'Total Sell']),cells=dict(values=[stock_name,quantity,buy_value,current_value,total_buy,total_sell]))])
                fig.show() 
        elif choice == 2 : 
            cur.execute( "SELECT SUM( CURRENT_TOTAL ) FROM INVESTMENT")
            val = cur.fetchone()
            cur.execute( "SELECT * FROM INVESTMENT ORDER BY CURRENT_TOTAL DESC" )
            arr = cur.fetchall( )
            cnt = 0 
            for x in arr :
                if x[ 1 ] > 0 : 
                    cnt += 1 
            if cnt == 0 : 
                print( 'NO STOCK AVAILABLE')
            else:
                stock =[]
                percentage = []
                for x in arr :
                    stock.append( x[ 0 ] )
                    percentage.append( round(( x[ 3 ] / val[ 0 ] ) * 100 , 3 ) )
                fig = tab.Figure(data=[tab.Table(columnwidth=[3,2],header=dict(values=['Stock Name','Weightage']),cells=dict(values=[stock,percentage]))])
                fig.show()
        elif( choice == 3 ):
            cur.execute( "SELECT SUM( Total_Buy ) FROM INVESTMENT " )
            Sum = cur.fetchone( )
            if( Sum[ 0 ] == None ):
                print( "EMPTY PORTFOLIO" ) 
            else :
                print( "TOTAL INVESTMENT :" , Sum[ 0 ] )
            cur.execute( "SELECT SUM( Total_Sell ) FROM INVESTMENT " )
            liquidity = cur.fetchone( )
            print( "TOTAL LIQUIDITY  :" , liquidity[ 0 ] )
            cur.execute( " SELECT SUM( current_total ) FROM INVESTMENT " )
            current_total = cur.fetchone( )
            print( "CURRENT PORTFOLIO:", current_total[ 0 ] )
            print( "NET PROFIT/LOSS  :" , liquidity[ 0 ] - ( Sum[ 0 ] - current_total[ 0 ] ) )
        elif( choice == 4 ): 
            m = input("\nWANT TO BUY NEW STOCK PRESS 0 , FOR EXSISTING STOCK PRESS 1 \n" )
            if int( m ) == 1 :
                cur.execute("SELECT * FROM INVESTMENT " )
                arr = cur.fetchall( )
                if len( arr ) == 0 :
                    print( "NO STOCK IN PORTFOLIO " )
                else:
                    print( "WHICH STOCK YOU WANT TO BUY :\n" )
                    serial_number = []
                    stock = []
                    for i , x in zip( range(len(arr)), arr ) :
                        serial_number.append( i + 1 )
                        stock.append(x[ 0 ])
                    fig = tab.Figure(data=[tab.Table( columnwidth=[1,4], header = dict( values =[ 'Number' , 'Stock Name']),cells=dict(values=[serial_number,stock]))])
                    fig.show() 
                    t = int( input( ) )
                    if t > len(arr):
                        print( "INVALID OUTPUT")
                    else:
                        stock_name = arr[ t - 1 ][ 0 ]
                        quantity = arr[ t - 1 ][ 1 ]
                        buy_price = arr[ t - 1 ][ 2 ] 
                        avg_price = arr[ t - 1 ][ 3 ]
                        total_buy = arr[ t - 1 ][ 4 ]
                        q = float( input( "\nENTER QUANTITY\n" ) ) 
                        price = float( input( "\nENTER BUY PRICE\n" ) )
                        buy_price = ( avg_price + q * price ) / ( quantity + q )
                        avg_price = avg_price + q * price 
                        sql = f'UPDATE INVESTMENT SET BUY_PRICE = "{buy_price}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        sql = f'UPDATE INVESTMENT SET CURRENT_TOTAL = "{avg_price}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        quantity += q 
                        sql = f'UPDATE INVESTMENT SET QUANTITY = "{quantity}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        total_buy += ( q * price )  
                        sql = f'UPDATE INVESTMENT SET TOTAL_BUY = "{total_buy}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        print( 'BOUGHT SUCCESSFULLY') 
            else:
                stock_name = input("\nENTER STOCK NAME:\n" ) 
                stock_name = stock_name.upper( )
                quantity = input( "\nENTER Quantity\n" ) 
                price = input( ("\nENTER BUY PRICE\n" ) )
                value = float( quantity ) * float( price )  
                sql = f'INSERT INTO INVESTMENT VALUES( "{stock_name}" , "{quantity}" , "{price}" , "{value}" , "{value}" , 0 )'
                cur.execute( sql ) 
                db.commit( )
                print( 'BOUGHT SUCCESSFULLY') 
        elif choice == 5:
            cur.execute( " SELECT * FROM INVESTMENT " )
            arr = cur.fetchall()
            if( len( arr ) == 0 ):
                print( "\nNO STOCK IN PORTFOLIO\n")
            else:
                print( "WHICH STOCK YOU WANT TO BUY :\n" )
                serial_number = []
                stock = []
                for i , x in zip( range(len(arr)), arr ) :
                    serial_number.append( i + 1 )
                    stock.append(x[ 0 ])
                fig = tab.Figure(data=[tab.Table( columnwidth=[1,4], header = dict( values =[ 'Number' , 'Stock Name']),cells=dict(values=[serial_number,stock]))])
                fig.show()
                t = int( input( ) )
                if t > len( arr ):
                    print('\t\t\t--------------------WRONG INPUT-------------------')
                else:
                    stock_name = arr[ t - 1 ][ 0 ]
                    quantity = arr[ t - 1 ][ 1 ]
                    buy_price = arr[ t - 1 ][ 2 ] 
                    current_value = arr[ t - 1 ][ 3 ]
                    total_sell = arr[ t - 1 ][ 5 ]
                    q = int( input('ENTER QUANTITY') )
                    p = float( input('ENTER PRICE') )
                    value = q * p 
                    if( q <= quantity) :
                        quantity -= q 
                        total_sell += value 
                        current_value -= q * buy_price
                        sql = f'UPDATE INVESTMENT SET QUANTITY = "{quantity}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( ) 
                        sql = f'UPDATE INVESTMENT SET TOTAL_SELL = "{total_sell}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        sql = f'UPDATE INVESTMENT SET CURRENT_TOTAL = "{current_value}" WHERE STOCK_NAME = "{stock_name}" ' 
                        cur.execute( sql )
                        db.commit( )
                        print( 'SOLD SUCCESSFULLY' )
                    else:
                        print( '\t\t\t--------------------INVALID INPUT-----------------')
        elif choice == 6 :
            break 
        else:
            print("galat chal rha hai ")
if __name__ == "__main__":
    main()

			--------------------WELCOME TO INVESTMENT BOOK--------------------


ENTER YOUR CHOICE :
1.See Portfolio
2.Show Weightage
3.Investment insight
4.Buy Stock
5.Sell Stock
6.Exit

5
WHICH STOCK YOU WANT TO BUY :



10
ENTER QUANTITY150
ENTER PRICE28.2
SOLD SUCCESSFULLY

ENTER YOUR CHOICE :
1.See Portfolio
2.Show Weightage
3.Investment insight
4.Buy Stock
5.Sell Stock
6.Exit

1



ENTER YOUR CHOICE :
1.See Portfolio
2.Show Weightage
3.Investment insight
4.Buy Stock
5.Sell Stock
6.Exit

3
TOTAL INVESTMENT : 139277.20993041992
TOTAL LIQUIDITY  : 4230.0
CURRENT PORTFOLIO: 135587.20993041992
NET PROFIT/LOSS  : 540.0

ENTER YOUR CHOICE :
1.See Portfolio
2.Show Weightage
3.Investment insight
4.Buy Stock
5.Sell Stock
6.Exit



###### cur.execute("CREATE TABLE INVESTMENT( STOCK_NAME VARCHAR( 20 ) PRIMARY KEY , QUANTITY INT , BUY_PRICE FLOAT , CURRENT_TOTAL FLOAT , TOTAL_BUY FLOAT , TOTAL_SELL FLOAT)")